### 1.Setup & Data Loading

In [10]:
import pandas as pd
import numpy as np
import os

# 1. CONFIGURATION
# We use the standard naming as requested
INPUT_FILENAME = "triple_barrier_12h.parquet" 
INPUT_PATH = os.path.join('../data', INPUT_FILENAME)

BUY_OUTPUT_FILENAME = 'features_buy_12h.parquet'
SELL_OUTPUT_FILENAME = 'features_sell_12h.parquet'

# 2. LOAD DATA
if os.path.exists(INPUT_PATH):
    print(f"⏳ Loading data from {INPUT_PATH}...")
    df = pd.read_parquet(INPUT_PATH)
else:
    print(f"❌ Error: {INPUT_PATH} not found. Run Notebook 2 first.")
    # Stop execution if running interactively
    raise FileNotFoundError("Data file missing.")

# Ensure index is datetime
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# 3. STANDARDIZE TIMEZONE (UTC+2 -> UTC)
# Crucial for Session features to align with London/NY opens
print("🌍 Shifting time from UTC+2 (Broker) to UTC...")
df.index = df.index - pd.Timedelta(hours=2)

# Verify
print(f"New start time (UTC): {df.index[0]}")
print(f"New end time (UTC):   {df.index[-1]}")
print(f"Initial Shape: {df.shape}")

⏳ Loading data from ../data\triple_barrier_12h.parquet...
🌍 Shifting time from UTC+2 (Broker) to UTC...
New start time (UTC): 2020-01-06 06:00:00
New end time (UTC):   2026-01-02 06:00:00
Initial Shape: (9332, 12)


### Helper Functions

In [11]:
# ==========================================
# HELPER FUNCTIONS
# ==========================================
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/period, adjust=False).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_atr(df, period=14):
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return tr.ewm(alpha=1/period, adjust=False).mean()

def calculate_psar(df, af_start=0.02, af_inc=0.02, af_max=0.2):
    # Iterative PSAR calculation
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values
    psar = np.zeros(len(df))
    trend = 1 if close[0] > close[1] else -1 
    af = af_start
    ep = high[0] if trend == 1 else low[0]
    psar[0] = low[0] if trend == 1 else high[0]
    
    for i in range(1, len(df)):
        prev_psar = psar[i-1]
        psar[i] = prev_psar + af * (ep - prev_psar)
        
        if trend == 1:
            psar[i] = min(psar[i], low[i-1])
            if i > 1: psar[i] = min(psar[i], low[i-2])
        else:
            psar[i] = max(psar[i], high[i-1])
            if i > 1: psar[i] = max(psar[i], high[i-2])
            
        reverse = False
        if trend == 1 and low[i] < psar[i]:
            trend = -1
            psar[i] = ep
            ep = low[i]
            af = af_start
            reverse = True
        elif trend == -1 and high[i] > psar[i]:
            trend = 1
            psar[i] = ep
            ep = high[i]
            af = af_start
            reverse = True
        
        if not reverse:
            if trend == 1:
                if high[i] > ep:
                    ep = high[i]
                    af = min(af + af_inc, af_max)
            else:
                if low[i] < ep:
                    ep = low[i]
                    af = min(af + af_inc, af_max)
                    
    return pd.Series(psar, index=df.index)

print("✅ Helper functions defined.")

✅ Helper functions defined.


### Feature Generation (The "Dream Team")

In [12]:
df_feat = df.copy()

print("🛠 Generating Features...")

# ==========================================
# 1. TIME CONTEXT (The "When")
# ==========================================
# Cyclical Clock
df_feat['hour_sin'] = np.sin(2 * np.pi * df_feat.index.hour / 24)
df_feat['hour_cos'] = np.cos(2 * np.pi * df_feat.index.hour / 24)

# Critical Sessions (London: 7-16, NY: 12-21, Tokyo: 0-9)
h = df_feat.index.hour
df_feat['sess_london'] = ((h >= 7) & (h <= 16)).astype(int)
df_feat['sess_ny'] = ((h >= 12) & (h <= 21)).astype(int)
df_feat['sess_tokyo'] = ((h >= 0) & (h <= 9)).astype(int)

# Overlaps (High Volatility Zones)
df_feat['sess_london_ny'] = (df_feat['sess_london'] & df_feat['sess_ny']).astype(int)

# ==========================================
# 2. VOLATILITY & FLOW (The "Fuel")
# ==========================================
# Volume Handling
vol_col = 'tick_volume' if 'tick_volume' in df_feat.columns else 'volume'
df_feat['volume'] = df_feat[vol_col]

# Relative Volume
df_feat['vol_rel'] = df_feat['volume'] / df_feat['volume'].rolling(20).mean()

# Spread Shock (If available)
if 'spread' in df_feat.columns:
    df_feat['spread_shock'] = df_feat['spread'] / df_feat['spread'].rolling(20).mean()
else:
    df_feat['spread_shock'] = 1.0 # Fallback

# NATR (Normalized ATR)
df_feat['ATR_14'] = calculate_atr(df_feat)
df_feat['NATR'] = (df_feat['ATR_14'] / df_feat['close']) * 100

# OBV (On Balance Volume)
df_feat['OBV'] = (np.sign(df_feat['close'].diff()) * df_feat['volume']).fillna(0).cumsum()

# ==========================================
# 3. TREND & SIGNAL (The "Trigger")
# ==========================================
# MACD Signal
ema12 = df_feat['close'].ewm(span=12, adjust=False).mean()
ema26 = df_feat['close'].ewm(span=26, adjust=False).mean()
df_feat['MACD'] = ema12 - ema26
df_feat['MACD_Signal'] = df_feat['MACD'].ewm(span=9, adjust=False).mean()

# Distance to PSAR
df_feat['PSAR'] = calculate_psar(df_feat)
df_feat['dist_psar'] = (df_feat['close'] - df_feat['PSAR']) / df_feat['close']

# Distance to EMA 200
df_feat['EMA_200'] = df_feat['close'].ewm(span=200, adjust=False).mean()
df_feat['dist_ema_200'] = (df_feat['close'] - df_feat['EMA_200']) / df_feat['EMA_200']

# # Distance to EMA 50 -- trial
df_feat['EMA_50'] = df_feat['close'].ewm(span=50, adjust=False).mean()
df_feat['dist_ema_50'] = (df_feat['close'] - df_feat['EMA_50']) / df_feat['EMA_50']

# RSI
df_feat['RSI_14'] = calculate_rsi(df_feat['close'])

print("✅ Feature Calculation Complete.")

🛠 Generating Features...
✅ Feature Calculation Complete.


In [13]:
df_feat.columns

Index(['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'h-l', 'h-pc',
       'l-pc', 'tr', 'atr', 'label', 'hour_sin', 'hour_cos', 'sess_london',
       'sess_ny', 'sess_tokyo', 'sess_london_ny', 'volume', 'vol_rel',
       'spread_shock', 'ATR_14', 'NATR', 'OBV', 'MACD', 'MACD_Signal', 'PSAR',
       'dist_psar', 'EMA_200', 'dist_ema_200', 'EMA_50', 'dist_ema_50',
       'RSI_14'],
      dtype='object')

### Selection & Cleanup

In [15]:
# ==========================================
# SELECT FEATURES
# ==========================================
keep_features = [
    # Context
    'sess_london_ny', 'sess_tokyo',
    
    # Volatility/Flow
    'NATR', 'vol_rel', 'OBV', 'spread_shock',
    
    # Trend/Signal
    'MACD_Signal', 'dist_psar', 'dist_ema_200', 'RSI_14'
]

# We include 'label' temporarily to create targets
cols_needed = keep_features + ['label']
df_dream = df_feat[cols_needed].copy()

# Drop Warmup Data (NaNs from EMA 200)
orig_len = len(df_dream)
df_dream.dropna(inplace=True)
print(f"Dropped {orig_len - len(df_dream)} warmup rows.")

# ==========================================
# CREATE SEPARATE DATASETS
# ==========================================

# 1. BUY DATASET
# Target = 1 (Buy). Everything else (Sell/Hold) = 0
df_buy = df_dream[keep_features].copy()
df_buy['target'] = (df_dream['label'] == 1).astype(int)

# 2. SELL DATASET
# Target = -1 (Sell). Everything else (Buy/Hold) = 0
df_sell = df_dream[keep_features].copy()
df_sell['target'] = (df_dream['label'] == -1).astype(int)

# Check Balance
print("\n📊 Final Class Balance:")
print(f"Buy Model:  {df_buy['target'].mean():.2%} Positives")
print(f"Sell Model: {df_sell['target'].mean():.2%} Positives")

Dropped 19 warmup rows.

📊 Final Class Balance:
Buy Model:  31.99% Positives
Sell Model: 31.62% Positives


### Save Datasets

In [16]:
OUTPUT_BUY = os.path.join('../data',BUY_OUTPUT_FILENAME)
OUTPUT_SELL = os.path.join('../data', SELL_OUTPUT_FILENAME)

# Save
df_buy.to_parquet(OUTPUT_BUY)
df_sell.to_parquet(OUTPUT_SELL)

print("💾 Datasets Saved Successfully:")
print(f"   -> Buy Model Data:  {OUTPUT_BUY}")
print(f"   -> Sell Model Data: {OUTPUT_SELL}")

💾 Datasets Saved Successfully:
   -> Buy Model Data:  ../data\features_buy_12h.parquet
   -> Sell Model Data: ../data\features_sell_12h.parquet


In [ ]:
# 1. Trend Indicators (Baselines)
# These indicators identify the general direction of the market and help traders "follow the trend". 
# Simple Moving Average (SMA): The average price over a set number of periods.
# Exponential Moving Average (EMA): Similar to SMA but weights recent prices more heavily.
# Weighted Moving Average (WMA): Assigns heavier weight to the most recent data points.
# Hull Moving Average (HMA): Designed to reduce lag and improve smoothness.
# Kaufman Adaptive Moving Average (KAMA): Adjusts its sensitivity based on market noise.
# Ichimoku Cloud (Kinko Hyo): A comprehensive system showing trend, support, and resistance.
# Parabolic SAR (Stop and Reverse): Uses dots to indicate potential trend reversals.
# Average Directional Index (ADX): Measures the strength of a trend.
# Aroon Indicator: Identifies when a trend is starting or changing.
# Aroon Oscillator: Measures the difference between Aroon Up and Aroon Down.
# Linear Regression Trendline: A straight line that best fits a set of price data.
# Double Exponential Moving Average (DEMA): Further reduces lag from standard EMAs.
# Triple Exponential Moving Average (TEMA): Uses triple smoothing to minimize lag.
# Supertrend: A trend-following indicator based on ATR.
# ZigZag: Filters out smaller price movements to show significant trends.
# Schaff Trend Cycle (STC): Combines MACD with a stochastic for faster trend signals.
# TRIX: A triple-smoothed exponential oscillator for trend identification.
# Detrended Price Oscillator (DPO): Removes trend to highlight short-term cycles.
# Alligator (Bill Williams): Uses three smoothed moving averages to identify trends.
# Gann Fans/Gann Lines: Based on geometric angles of price and time. 

# 2. Momentum Indicators (Oscillators)
# These measure the speed of price movements and identify overbought or oversold conditions.
# 21. Relative Strength Index (RSI): Ranges 0-100; signals overbought (>70) or oversold (<30).
# 22. Stochastic Oscillator: Compares closing price to its range over time.
# 23. MACD (Moving Average Convergence Divergence): Shows the relationship between two EMAs.
# 24. Commodity Channel Index (CCI): Identifies cyclical trends and reversals.
# 25. Williams %R: Shows where the current price is relative to the highest high.
# 26. Awesome Oscillator (AO): Measures market momentum using 34 and 5-period SMAs.
# 27. Momentum Indicator: Measures the rate of change of prices.
# 28. Rate of Change (ROC): Calculates the percentage change in price between periods.
# 29. Money Flow Index (MFI): Volume-weighted version of RSI.
# 30. Relative Vigor Index (RVI): Measures the strength of a trend by comparing closing prices.
# 31. Stochastic RSI: A stochastic applied to RSI values for increased sensitivity.
# 32. Ultimate Oscillator: Uses three different timeframes to reduce false signals.
# 33. Chande Momentum Oscillator (CMO): Calculates momentum based on unsmoothed data.
# 34. Gator Oscillator: Derived from the Alligator indicator to show trend changes.
# 35. DeMarker (DeM): Compares most recent maximum and minimum prices.
# 36. True Strength Index (TSI): A double-smoothed momentum oscillator.
# 37. Vortex Indicator: Two lines (+VI and -VI) that identify the start of a new trend.
# 38. Fisher Transform: Transforms prices into a Gaussian normal distribution.
# 39. Center of Gravity Oscillator: Identifies major turning points without lag.
# 40. Percentage Price Oscillator (PPO): Similar to MACD but shown in percentages. 

# 3. Volatility Indicators
# These measure how far price stretches from its mean, helping with risk management and breakout detection.
# 41. Bollinger Bands: A moving average with two standard deviation bands.
# 42. Average True Range (ATR): Measures the average range of price movement.
# 43. Standard Deviation: Measures how spread out price data is from the mean.
# 44. Keltner Channels: Volatility-based envelopes set above/below an EMA.
# 45. Donchian Channels: Shows the highest high and lowest low over a period.
# 46. Envelopes: Two moving averages set at a fixed percentage above and below price.
# 47. Chaikin Volatility: Measures the difference between high and low prices.
# 48. Bollinger Bandwidth: Measures the distance between upper and lower Bollinger Bands.
# 49. STARC Bands: Combines moving averages and ATR to create volatility channels.
# 50. Ulcer Index: Measures "stress" by analyzing the depth and duration of price drops.
# 51. Relative Volatility Index (RVI): Measures the direction of volatility.
# 52. Choppiness Index: Determines if the market is trending or "choppy".
# 53. Mass Index: Predicts reversals by measuring the narrowing/widening of price ranges.
# 54. Historical Volatility (HV): Measures the past standard deviation of an asset. 
    
#     4. Volume & Support/Resistance Indicators
# These measure market participation and key price levels.
# 55. On-Balance Volume (OBV): Relates volume to price change to confirm trends.
# 56. Accumulation/Distribution (A/D): Measures the cumulative flow of money.
# 57. Chaikin Money Flow (CMF): Measures the amount of Money Flow Volume over a period.
# 58. Volume Weighted Average Price (VWAP): Average price weighted by total volume.
# 59. Pivot Points (Standard): Key levels calculated from the previous day's H/L/C.
# 60. Fibonacci Retracements: Horizontal lines based on Fibonacci ratios to find support.
# 61. Volume Profile: Shows volume traded at specific price levels.
# 62. Ease of Movement (EOM): Relates price change to volume.
# 63. Money Flow Ratio: Compares positive and negative money flow.
# 64. Negative Volume Index (NVI): Focuses on days where volume decreased.
# 65. Positive Volume Index (PVI): Focuses on days where volume increased.
# 66. Price Volume Trend (PVT): Cumulative volume that adds a percentage of the day's volume.
# 67. Trade Volume Index (TVI): Used to determine whether an asset is being accumulated.
# 68. Market Profile: Visualizes price and volume over time (TPO).
# 69. Balance of Power (BOP): Measures the strength of buyers vs. sellers.
# 70. Camarilla Pivot Points: A variation of pivots providing tighter support/resistance.
# 71. Woodie’s Pivot Points: Weighted differently to emphasize recent price action.
# 72. Fibonacci Extensions: Projects potential future profit-taking levels.
# 73. Elder Force Index: Combines price movement and volume to measure trend power.
# 74. Vortex Indicator (VI): Identifies the start of a trend based on high/low distance.
# 75. Chaikin Oscillator: Applies MACD to the Accumulation/Distribution line.
# 76. Force Index: Uses price, volume, and time to identify trend strength.
# 77. Klinger Oscillator: Compares volume flowing through an asset with price

# 5. Specialized & Hybrid Indicators
# Fractals (Bill Williams): Arrows that highlight local high and low points.
# Heiken Ashi: Specialized candlesticks that filter market noise.
# TD Sequential: Identifies trend exhaustion and potential price flips.
# Murrey Math Lines: Support/resistance levels based on Gann's theory.
# Psychological Line: Measures the ratio of rising days to total days.
# Volume Rate of Change (VROC): Percentage change in volume.
# Typical Price: (High + Low + Close) / 3.
# Median Price: (High + Low) / 2.
# Adaptive Lagging Line: Follows price but smooths out data without excessive lag.
# Trend Intensity Index (TII): Measures the strength of a trend.
# Rainbow Moving Average: Multiple MAs used together to show trend maturity.
# Linear Regression Slope: Measures the rate of change of a linear regression line.
# CVI (Chart Volatility Index): A custom measure of intraday volatility.
# Gpivot Ressup: Dynamic support and resistance pivot variation.
# DMI (Directional Movement Index): Consists of +DI and -DI lines.
# TMA (Triangular Moving Average): Double-smoothed version of the SMA.
# QQE (Quantitative Qualitative Estimation): Smother RSI-based indicator.
# RSX (Relative Strength Index smoothed): A noise-free version of RSI.
# ATR Trailing Stop: A stop-loss level that moves with the price.
# Connors RSI: Combines RSI, Up/Down Length, and Rate of Change.
# Forex Sentiment: Measures the percentage of long vs. short retail positions.
# Currency Strength Meter: Compares one currency's performance against all others.
# Market Sessions Indicator: Highlights Asian, London, and New York trading hours. 



